In [18]:
from pathlib import Path
from label_studio_sdk.client import LabelStudio
from label_studio_sdk.core import ApiError
from fishsense_api_sdk.client import Client
import asyncio
from tqdm.asyncio import tqdm_asyncio
from tqdm.notebook import tqdm
import json
from fishsense_api_sdk.models.laser_label import LaserLabel

In [19]:
URL = "http://localhost:8000"

In [20]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [21]:
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [22]:
async with Client(URL) as fs:
    dives = await fs.dives.get_canonical()

len(dives), dives[0]

(272,
 Dive(id=1, name='080123_FSL-01 Photos', path='2023-09-07 REEF Data Dump/080123_FSL-01 Photos', dive_datetime=datetime.datetime(2023, 8, 1, 12, 46, 27, tzinfo=TzInfo(0)), priority=<Priority.LOW: 'LOW'>, flip_dive_slate=True, camera_id=1, dive_slate_id=1))

In [23]:
async with Client(URL) as fs:
    laser_labels = await tqdm_asyncio.gather(*[fs.labels.get_laser_labels(dive.id) for dive in dives])
    laser_labels = [label for sublist in laser_labels for label in sublist]

len(laser_labels), laser_labels[0] 

100%|██████████| 272/272 [00:09<00:00, 27.82it/s]


(44563,
 LaserLabel(id=56077, label_studio_task_id=33678, label_studio_project_id=10, x=1909.8534480774288, y=1262.9521714239743, label='Red Laser', updated_at=datetime.datetime(2024, 11, 19, 21, 45, 32, 955706, tzinfo=TzInfo(0)), superseded=False, completed=True, label_studio_json={'id': 33678, 'annotations': [{'id': 8586, 'completed_by': 8, 'result': [{'id': 'NJV60yCBLC', 'type': 'keypointlabels', 'value': {'x': 47.9020177596546, 'y': 41.875071996816125, 'width': 0.20145365914588592, 'keypointlabels': ['Red Laser']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 3987, 'original_height': 3016}], 'was_cancelled': False, 'ground_truth': False, 'created_at': '2024-11-19T21:45:32.903793Z', 'updated_at': '2024-11-19T21:45:32.903805Z', 'draft_created_at': None, 'lead_time': 5.091, 'prediction': {}, 'result_count': 0, 'unique_id': '84e720cf-f3f0-4f49-97b7-6e39e4013c32', 'import_id': None, 'last_action': None, 'task': 33678, 'project': 10,

In [24]:
label_studio_project_ids = list({label.label_studio_project_id for label in laser_labels if label.label_studio_project_id is not None})

len(label_studio_project_ids), label_studio_project_ids

(10, [10, 43, 42, 57, 58, 59, 60, 61, 62, 63])

In [25]:
tasks = []

for project_id in tqdm(label_studio_project_ids):
    try:
        project = ls.projects.get(project_id)
    except ApiError as e:
        print(f"Error fetching project {project_id}: {e}")
        continue

    tasks += list(ls.tasks.list(project=project_id))

len(tasks), tasks[0]

  0%|          | 0/10 [00:00<?, ?it/s]

Error fetching project 10: status_code: 404, body: {'id': '0e93c1be-aa64-48ab-b28d-221c0b1484b3', 'status_code': 404, 'version': '1.21.0', 'detail': 'No Project matches the given query.', 'exc_info': None}


(30019,
 LseTask(agreement=None, agreement_selected=None, annotations=[{'id': 92210, 'result': [{'id': 'ieVgIKA91P', 'type': 'keypointlabels', 'value': {'x': 49.00506942030147, 'y': 43.05559796746316, 'width': 0.2757321850587165, 'keypointlabels': ['Red Laser']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}], 'created_username': 'Kshitij Khandagale kshitijkhandagale2007@gmail.com, 131', 'created_ago': '6\xa0months', 'completed_by': 131, 'was_cancelled': False, 'ground_truth': False, 'created_at': '2025-05-21T18:03:55.312289Z', 'updated_at': '2025-05-21T18:03:55.312310Z', 'draft_created_at': '2025-05-21T18:03:53.038156Z', 'lead_time': 52.045, 'import_id': None, 'last_action': None, 'bulk_created': False, 'task': 190321, 'project': 43, 'updated_by': 131, 'parent_prediction': None, 'parent_annotation': None, 'last_created_by': None}], annotations_ids='92210', annotations_results='[{id:ieVgIKA91P, type:ke

In [26]:
label_studio_task_to_laser_label_map = {
    label.label_studio_task_id: label
    for label in laser_labels
}

len(label_studio_task_to_laser_label_map)

44563

In [27]:
LASER_LABEL_KEY_NAMES = ["kp-1", "laser"]

coroutines = []
async with Client(URL) as fs:
    for task in tqdm(tasks):
        laser_label: LaserLabel = label_studio_task_to_laser_label_map.get(task.id)

        if laser_label is None:
            continue

        email = task.annotations[0]["created_username"].split(",")[0].split(' ')[-1].strip()
        tqdm.write(f"Processing task {task.id} for user {email}")
        user = await fs.users.get(email=email)

        laser_label.label_studio_json = json.loads(task.json())
        laser_label.user_id = user.id
        laser_label.updated_at = task.updated_at
        laser_label.completed = task.is_labeled

        if len(task.annotations) > 0:
            for laser_label_key in LASER_LABEL_KEY_NAMES:
                laser_label_section = [r for r in task.annotations[0]["result"] if r["from_name"] == laser_label_key]
                laser_label_section = laser_label_section[0] if len(laser_label_section) > 0 else None

                if laser_label_section is not None:
                    break

            if laser_label_section is not None:
                original_width = laser_label_section["original_width"]
                original_height = laser_label_section["original_height"]

                laser_x = laser_label_section["value"]["x"] * original_width / 100
                laser_y = laser_label_section["value"]["y"] * original_height / 100
                laser_label_str = laser_label_section["value"]["keypointlabels"][0]

                laser_label.x = laser_x
                laser_label.y = laser_y
                laser_label.label = laser_label_str
        
        coroutines.append(fs.labels.put_laser_label(laser_label.image_id, laser_label))

    await tqdm_asyncio.gather(*coroutines)

  0%|          | 0/30019 [00:00<?, ?it/s]

Processing task 190321 for user kshitijkhandagale2007@gmail.com
Processing task 190322 for user kshitijkhandagale2007@gmail.com
Processing task 190323 for user kshitijkhandagale2007@gmail.com
Processing task 190324 for user kshitijkhandagale2007@gmail.com
Processing task 190325 for user kshitijkhandagale2007@gmail.com
Processing task 190326 for user kshitijkhandagale2007@gmail.com
Processing task 190327 for user jessarmacsot@gmail.com
Processing task 190328 for user kshitijkhandagale2007@gmail.com
Processing task 190329 for user kshitijkhandagale2007@gmail.com
Processing task 190330 for user kshitijkhandagale2007@gmail.com
Processing task 190331 for user kshitijkhandagale2007@gmail.com
Processing task 190332 for user kshitijkhandagale2007@gmail.com
Processing task 190333 for user taylorvh2000@gmail.com
Processing task 190334 for user kshitijkhandagale2007@gmail.com
Processing task 190335 for user kshitijkhandagale2007@gmail.com
Processing task 190336 for user kshitijkhandagale2007@gmai

100%|██████████| 8249/8249 [01:09<00:00, 119.45it/s]


In [28]:
task.annotations[0]

{'id': 122097,
 'result': [{'id': 'laser_result',
   'type': 'keypointlabels',
   'value': {'x': 50.54808171400099,
    'y': 47.97745358090186,
    'width': 0.2,
    'keypointlabels': ['Green Laser']},
   'origin': 'prediction',
   'to_name': 'image',
   'from_name': 'laser',
   'image_rotation': 0,
   'original_width': 4014,
   'original_height': 3016},
  {'id': 'yH8kcnJfPf',
   'type': 'choices',
   'value': {'choices': ['Top 3 photos of group']},
   'origin': 'manual',
   'to_name': 'image',
   'from_name': 'exclude'},
  {'id': 'f3B_Jcw2W2',
   'type': 'taxonomy',
   'value': {'taxonomy': [['Fish', 'Hogfish (Lachnolaimus maximus)']]},
   'origin': 'manual',
   'to_name': 'image',
   'from_name': 'species'},
  {'id': 'wqJ7jCKAAA',
   'type': 'taxonomy',
   'value': {'taxonomy': [['yes, center of fish']]},
   'origin': 'manual',
   'to_name': 'image',
   'from_name': 'measurable'},
  {'id': 'PkupM8HNMB',
   'type': 'taxonomy',
   'value': {'taxonomy': [['x < 5°']]},
   'origin': 'manu

In [29]:

laser_label_section = [r for r in task.annotations[0]["result"] if r["from_name"] == "laser"]
laser_label_section = laser_label_section[0] if len(laser_label_section) > 0 else None

laser_label_section

{'id': 'laser_result',
 'type': 'keypointlabels',
 'value': {'x': 50.54808171400099,
  'y': 47.97745358090186,
  'width': 0.2,
  'keypointlabels': ['Green Laser']},
 'origin': 'prediction',
 'to_name': 'image',
 'from_name': 'laser',
 'image_rotation': 0,
 'original_width': 4014,
 'original_height': 3016}